## ⏬ **Import Modules & Set Up Environment**

In [1]:
import pandas as pd  # For working with dataframes and performing data manipulation
import re  # For handling regular expressions

# Importing additional functionality from IPython
from IPython.core.interactiveshell import InteractiveShell

# Setting an IPython display option to show all outputs in a single cell
# By default, Jupyter only shows the result of the last line in a cell.
# This line changes that behavior to display the output of all statements in the cell.
InteractiveShell.ast_node_interactivity = "all"

## 📂 **Load Dataset**

In [2]:
# Read data
df = pd.read_csv("car_data.csv")

## 🔎 **Explore Data**

In [3]:
# Check number of rows and columns
df.shape

(4411, 19)

In [4]:
# Show first 3 rows
df.head(3)

,Brand,Type,Reg_date,Coe_left,Dep,Mileage,Road Tax,Dereg Value,COE,Engine Cap,Curb Weight,Manufactured,Transmission,OMV,ARF,Power,No. of Owners,Price,Unnamed: 18
0,BMW 5 Series 520i Highline,Luxury Sedan,28-May-13,1yr 11mths 21days COE left),15920,N.A,1210,38133,67304,1997,1610,2013,Auto,41566,45193,135,1,54000,NaN
1,Honda Vezel 1.5A X Honda Sensing,SUV,14-Oct-15,4yrs 4mths 7days COE left),10310,50000,682,31237,56001,1496,1190,2015,Auto,19775,9775,96,2,49800,NaN
2,MINI One 1.6A (New 10-yr COE),Hatchback,7-Oct-11,10yrs COE left),6280,110000,742,N.A,N.A,1598,1100,2011,Auto,18180,18180,72,3,62800,NaN


In [5]:
# Display a summary of the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4411 entries, 0 to 4410
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Brand          4411 non-null   object 
 1   Type           4411 non-null   object 
 2   Reg_date       4411 non-null   object 
 3   Coe_left       4389 non-null   object 
 4   Dep            4411 non-null   object 
 5   Mileage        4411 non-null   object 
 6   Road Tax       4411 non-null   object 
 7   Dereg Value    4411 non-null   object 
 8   COE            4411 non-null   object 
 9   Engine Cap     4411 non-null   object 
 10  Curb Weight    4411 non-null   object 
 11  Manufactured   4411 non-null   object 
 12  Transmission   4411 non-null   object 
 13  OMV            4411 non-null   object 
 14  ARF            4411 non-null   object 
 15  Power          4411 non-null   object 
 16  No. of Owners  4411 non-null   object 
 17  Price          4411 non-null   object 
 18  Unnamed:

In [6]:
# Count the total number of duplicate values in the 'Brand' column
num_duplicates = df['Brand'].duplicated(keep=False).sum()

# Count the total number of unique values in the 'Brand' column
num_unique = df['Brand'].nunique()

# Display the results
print(f"Number of duplicate values: {num_duplicates}")
print(f"Number of unique values: {num_unique}")

Number of duplicate values: 2840
Number of unique values: 2159


## 🧹 **Data Cleaning**

### **Step 1: Check for unnamed columns and drop them**

In [7]:
# Identify unnamed columns
# - Create a list of columns that contain the substring 'Unnamed' in their names.
# - These columns are typically created when reading a CSV file with an index column.
unnamed_columns = [col for col in df.columns if 'Unnamed' in col]

# Check if unnamed columns exist
if unnamed_columns:
    # If unnamed columns are found, print a message and drop them
    print(f"Found unnamed columns: {unnamed_columns}. Dropping them.")
    df.drop(columns=unnamed_columns, inplace=True)  # Drop the unnamed columns
else:
    # If no unnamed columns are found, print a message
    print("No unnamed columns found.")

# Check the shape of the DataFrame after cleaning
df.shape

Found unnamed columns: ['Unnamed: 18']. Dropping them.


(4411, 18)

### **Step 2: Remove rows with empty or NA values**

In [8]:
# Replace 'N.A', 'NA', 'N.A.' with NaN (missing value indicator)
# - This ensures that all instances of 'N.A', 'NA', and 'N.A.' in the DataFrame are treated as missing values.
df.replace(['N.A', 'NA', 'N.A.'], pd.NA, inplace=True)

# Remove rows with any missing values (NaN)
# - The `how='any'` parameter ensures that rows with **any** missing values are dropped.
# - The `inplace=True` parameter modifies the DataFrame in place, without creating a new DataFrame.
df.dropna(how='any', inplace=True)

# Check the shape of the DataFrame after cleaning
df.shape

(2494, 18)

### **Step 3: Convert values in `Reg_date` to datetime**

In [9]:
# Display the first row from the DataFrame to inspect the data before transformation
print("Before:")
df.head(1)

# Convert the 'Reg_date' column to datetime format
# - '%d-%b-%y' specifies the date format: day-month-year (e.g., 28-May-13)
# - 'errors="coerce"' ensures invalid dates are converted to NaT (Not a Time) instead of raising an error
df['Reg_date'] = pd.to_datetime(df['Reg_date'], format='%d-%b-%y', errors='coerce')

# Extract year, month, and day from 'Reg_date' and create new features
df['Reg_year'] = df['Reg_date'].dt.year
df['Reg_month'] = df['Reg_date'].dt.month
# df['Reg_day'] = df['Reg_date'].dt.day # Not included - might add noise and have little impact on the model's performance

# Calculate the number of days since registration (from the current date)
df['Days_since_registration'] = (pd.to_datetime('today') - df['Reg_date']).dt.days

# Drop the original 'Reg_date' column
df = df.drop(columns=['Reg_date'])

# Reorder the columns to place the new features right after 'Reg_year' and 'Reg_month'
new_column_order = ['Brand', 'Type', 'Reg_year', 'Reg_month', 'Days_since_registration', 'Coe_left', 'Dep', 'Mileage', 'Road Tax', 'Dereg Value', 'COE', 'Engine Cap', 'Curb Weight', 'Manufactured', 'Transmission', 'OMV', 'ARF', 'Power', 'No. of Owners', 'Price']
df = df[new_column_order]

# Display the first row from the DataFrame to verify the transformation
print("After:")
df.head(1)


Before:


,Brand,Type,Reg_date,Coe_left,Dep,Mileage,Road Tax,Dereg Value,COE,Engine Cap,Curb Weight,Manufactured,Transmission,OMV,ARF,Power,No. of Owners,Price
1,Honda Vezel 1.5A X Honda Sensing,SUV,14-Oct-15,4yrs 4mths 7days COE left),10310,50000,682,31237,56001,1496,1190,2015,Auto,19775,9775,96,2,49800


After:


,Brand,Type,Reg_year,Reg_month,Days_since_registration,Coe_left,Dep,Mileage,Road Tax,Dereg Value,COE,Engine Cap,Curb Weight,Manufactured,Transmission,OMV,ARF,Power,No. of Owners,Price
1,Honda Vezel 1.5A X Honda Sensing,SUV,2015,10,3392,4yrs 4mths 7days COE left),10310,50000,682,31237,56001,1496,1190,2015,Auto,19775,9775,96,2,49800


### **Step 4: Convert text values in `Coe_left` to number of days**

In [10]:
# Function to convert 'Coe_left' text to number of days
def coe_left_to_days(text):
    # Handle missing values
    if pd.isna(text):
        return pd.NA
    
    # Extract years, months, and days using regex
    years_match = re.search(r'(\d+)yrs', text)  # Find years
    months_match = re.search(r'(\d+)mths', text)  # Find months
    days_match = re.search(r'(\d+)days', text)  # Find days
    
    # Convert matches to integers; default to 0 if not found
    years = int(years_match.group(1)) if years_match else 0
    months = int(months_match.group(1)) if months_match else 0
    days = int(days_match.group(1)) if days_match else 0
    
    # Calculate total days: years*365 + months*30 + days
    total_days = years * 365 + months * 30 + days
    return total_days

In [11]:
# Apply the function to the 'Coe_left' column and store results in 'Coe_left_days'
df['Coe_left_days'] = df['Coe_left'].apply(coe_left_to_days)

# Display a random sample of 3 rows to verify the transformation
df.sample(3)

,Brand,Type,Reg_year,Reg_month,Days_since_registration,Coe_left,Dep,Mileage,Road Tax,Dereg Value,...,Engine Cap,Curb Weight,Manufactured,Transmission,OMV,ARF,Power,No. of Owners,Price,Coe_left_days
4041,Mazda 6 2.5A Luxury,Luxury Sedan,2017,2,2893,5yrs 8mths 17days COE left),10410,57200,1786,48940,...,2488,1515,2016,Auto,26059,28483,141,1,73800,2082
3576,Audi A3 Sedan 1.0A TFSI S-tronic,Luxury Sedan,2018,3,2505,6yrs 9mths 12days COE left),11750,34600,392,47082,...,999,1280,2017,Auto,24702,26583,85,1,92999,2472
1456,Skoda Octavia 2.0A TSI RS,Mid-Sized Sedan,2019,10,1914,8yrs 4mths 24days COE left),13750,14000,1194,51129,...,1984,1577,2018,Auto,26300,28820,180,1,129999,3064


In [12]:
# Replace the 'Coe_left' column with the 'Coe_left_days' values
df['Coe_left'] = df['Coe_left_days']

# Drop the 'Coe_left_days' column as it's no longer needed
df.drop(columns=['Coe_left_days'], inplace=True)

# Display a random sample of 3 rows to verify the transformation
df.sample(3)

,Brand,Type,Reg_year,Reg_month,Days_since_registration,Coe_left,Dep,Mileage,Road Tax,Dereg Value,COE,Engine Cap,Curb Weight,Manufactured,Transmission,OMV,ARF,Power,No. of Owners,Price
1672,Mitsubishi Lancer EX 1.5A GLS (COE till 03/2024),Mid-Sized Sedan,2009,3,5799,1005,8610,143000,889,7056,12763,1499,1313,2009,Auto,19788,19788,80,3,23800
427,Honda Fit 1.3A G (COE till 02/2024),Hatchback,2009,2,5819,983,8790,150000,767,6964,12864,1339,1010,2008,Auto,14665,14665,73,5,23800
1937,Hyundai Accent 1.4A,Mid-Sized Sedan,2020,7,1642,3307,7560,32000,608,36826,32699,1368,1115,2019,Auto,9218,9218,73,1,73800


### **Step 5: Clean numeric columns**

(Precautionary measure)

In [13]:
# Display first 2 rows before transformation
print("Before:")
df.head(2)

# List of numeric columns to clean and convert
numeric_columns = ['Coe_left', 'Dep', 'Mileage', 'Road Tax', 'Dereg Value', 'COE', 'Engine Cap', 'Curb Weight', 'OMV', 'ARF', 'Power', 'Price']

# Loop through each column in the numeric_columns list
for col in numeric_columns:
    # Step 1: Remove non-numeric characters (e.g., $, commas)
    # - Convert the column to string type to ensure the str.replace() method works
    # - Use regex to remove all characters that are not digits (\d) or decimal points (.)
    df[col] = df[col].astype(str).str.replace(r'[^\d.]', '', regex=True)
    
    # Step 2: Convert the cleaned strings to numeric values
    # - Use pd.to_numeric() to convert the column to numeric type (int or float)
    # - Set errors='coerce' to replace invalid values (e.g., empty strings) with NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Display first 2 rows to verify the transformation
print("After:")
df.head(2)

Before:


,Brand,Type,Reg_year,Reg_month,Days_since_registration,Coe_left,Dep,Mileage,Road Tax,Dereg Value,COE,Engine Cap,Curb Weight,Manufactured,Transmission,OMV,ARF,Power,No. of Owners,Price
1,Honda Vezel 1.5A X Honda Sensing,SUV,2015,10,3392,1587,10310,50000,682,31237,56001,1496,1190,2015,Auto,19775,9775,96,2,49800
4,Suzuki Swift Sport 1.6M (COE till 12/2022),Hatchback,2007,12,6239,201,8210,203000,1030,6656,21349,1586,1060,2007,Manual,12154,13370,92,2,12800


After:


,Brand,Type,Reg_year,Reg_month,Days_since_registration,Coe_left,Dep,Mileage,Road Tax,Dereg Value,COE,Engine Cap,Curb Weight,Manufactured,Transmission,OMV,ARF,Power,No. of Owners,Price
1,Honda Vezel 1.5A X Honda Sensing,SUV,2015,10,3392,1587,10310,50000,682,31237,56001,1496,1190,2015,Auto,19775,9775,96,2,49800
4,Suzuki Swift Sport 1.6M (COE till 12/2022),Hatchback,2007,12,6239,201,8210,203000,1030,6656,21349,1586,1060,2007,Manual,12154,13370,92,2,12800


### **Step 6: Clean categorical columns**

(Precautionary measure)

In [14]:
# Display first 2 rows before transformation
print("Before:")
df.head(2)

# Define a list of column names that are categorical and need cleaning.
# The number of owners usually falls into a small, finite range (e.g., 1, 2, 3, etc.).
# These values represent distinct categories rather than continuous quantities. For example:
# A vehicle with "1 owner" is categorically different from a vehicle with "3 owners."
# There’s no meaningful interpolation between these numbers (e.g., "1.5 owners" doesn’t make sense).
categorical_columns = ['Brand', 'Type', 'Transmission', 'No. of Owners']

# Loop through each column in the list.
for col in categorical_columns:
    # For each column, clean the text data:
    # 1. `str.strip()` removes any leading or trailing whitespace from the text in each cell.
    # 2. `str.title()` converts the text to title case (e.g., "honda civic" becomes "Honda Civic").
    df[col] = df[col].str.strip().str.title()

# Display first 2 rows to verify the transformation
print("After:")
df.head(2)

Before:


,Brand,Type,Reg_year,Reg_month,Days_since_registration,Coe_left,Dep,Mileage,Road Tax,Dereg Value,COE,Engine Cap,Curb Weight,Manufactured,Transmission,OMV,ARF,Power,No. of Owners,Price
1,Honda Vezel 1.5A X Honda Sensing,SUV,2015,10,3392,1587,10310,50000,682,31237,56001,1496,1190,2015,Auto,19775,9775,96,2,49800
4,Suzuki Swift Sport 1.6M (COE till 12/2022),Hatchback,2007,12,6239,201,8210,203000,1030,6656,21349,1586,1060,2007,Manual,12154,13370,92,2,12800


After:


,Brand,Type,Reg_year,Reg_month,Days_since_registration,Coe_left,Dep,Mileage,Road Tax,Dereg Value,COE,Engine Cap,Curb Weight,Manufactured,Transmission,OMV,ARF,Power,No. of Owners,Price
1,Honda Vezel 1.5A X Honda Sensing,Suv,2015,10,3392,1587,10310,50000,682,31237,56001,1496,1190,2015,Auto,19775,9775,96,2,49800
4,Suzuki Swift Sport 1.6M (Coe Till 12/2022),Hatchback,2007,12,6239,201,8210,203000,1030,6656,21349,1586,1060,2007,Manual,12154,13370,92,2,12800


### **Step 7: Rename columns**

In [15]:
# Using underscores (_) instead of spaces (e.g., Road_Tax instead of Road Tax) makes it easier to reference columns in code.
df.rename(columns={
    'Dep': 'Depreciation',
    'Road Tax': 'Road_Tax',
    'Dereg Value': 'Dereg_Value',
    'Engine Cap': 'Engine_Capacity',
    'Curb Weight': 'Curb_Weight',
    'No. of Owners': 'Number_of_Owners'
}, inplace=True)

# Display new column names
print(df.columns)

Index(['Brand', 'Type', 'Reg_year', 'Reg_month', 'Days_since_registration',
       'Coe_left', 'Depreciation', 'Mileage', 'Road_Tax', 'Dereg_Value', 'COE',
       'Engine_Capacity', 'Curb_Weight', 'Manufactured', 'Transmission', 'OMV',
       'ARF', 'Power', 'Number_of_Owners', 'Price'],
      dtype='object')


### **Step 8: Validate data**

In [16]:
df.shape

(2494, 20)

In [17]:
df.head(3)

,Brand,Type,Reg_year,Reg_month,Days_since_registration,Coe_left,Depreciation,Mileage,Road_Tax,Dereg_Value,COE,Engine_Capacity,Curb_Weight,Manufactured,Transmission,OMV,ARF,Power,Number_of_Owners,Price
1,Honda Vezel 1.5A X Honda Sensing,Suv,2015,10,3392,1587,10310,50000,682,31237,56001,1496,1190,2015,Auto,19775,9775,96,2,49800
4,Suzuki Swift Sport 1.6M (Coe Till 12/2022),Hatchback,2007,12,6239,201,8210,203000,1030,6656,21349,1586,1060,2007,Manual,12154,13370,92,2,12800
5,Porsche 718 Cayman 2.0A Pdk,Sports Car,2017,7,2748,2202,34200,21000,1200,106829,50110,1988,1365,2017,Auto,71979,101563,220,1,259988


In [18]:
df.describe()

,Reg_year,Reg_month,Days_since_registration,Coe_left,Depreciation,Mileage,Road_Tax,Dereg_Value,COE,Engine_Capacity,Curb_Weight,OMV,ARF,Power,Price
count,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2.494000e+03
mean,2015.070970,6.726945,3460.803528,2156.439054,15137.919006,74854.008821,1210.225742,48025.403368,44216.838813,1894.682438,1444.451083,36057.646752,40291.493585,129.184443,1.001471e+05
std,3.846036,3.432639,1402.712090,872.800408,18149.236539,52207.367470,988.787188,40828.122719,16321.491449,712.548036,260.143161,39027.444860,56132.877889,67.709430,9.313203e+04
min,2005.000000,1.000000,1335.000000,3.000000,4900.000000,18.000000,70.000000,4036.000000,8000.000000,658.000000,806.000000,5588.000000,541.000000,40.000000,7.200000e+03
25%,2012.000000,4.000000,2403.000000,1620.250000,9032.500000,35000.000000,684.000000,30416.000000,32909.000000,1497.000000,1265.000000,18470.000000,15368.750000,90.000000,5.580000e+04
50%,2016.000000,7.000000,3085.500000,2201.500000,11345.000000,65255.500000,889.000000,39581.000000,42683.000000,1598.000000,1425.000000,26030.000000,25518.500000,109.500000,7.580000e+04
75%,2018.000000,10.000000,4422.750000,2850.000000,15485.000000,108000.000000,1210.000000,55113.500000,53927.750000,1998.000000,1610.000000,38196.000000,43656.750000,135.000000,1.158000e+05
max,2021.000000,12.000000,7199.000000,3640.000000,313020.000000,610000.000000,10611.000000,629526.000000,97889.000000,6752.000000,2760.000000,492350.000000,858230.000000,530.000000,1.138000e+06


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2494 entries, 1 to 4409
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Brand                    2494 non-null   object
 1   Type                     2494 non-null   object
 2   Reg_year                 2494 non-null   int32 
 3   Reg_month                2494 non-null   int32 
 4   Days_since_registration  2494 non-null   int64 
 5   Coe_left                 2494 non-null   int64 
 6   Depreciation             2494 non-null   int64 
 7   Mileage                  2494 non-null   int64 
 8   Road_Tax                 2494 non-null   int64 
 9   Dereg_Value              2494 non-null   int64 
 10  COE                      2494 non-null   int64 
 11  Engine_Capacity          2494 non-null   int64 
 12  Curb_Weight              2494 non-null   int64 
 13  Manufactured             2494 non-null   object
 14  Transmission             2494 non-null   obje

### **Step 9: Save cleaned dataset**

In [20]:
df.to_csv('cleaned_car_data.csv', index=False)

# ----------------------------------- END -----------------------------------